In [53]:
import ee
import numpy as np
import pandas as pd

from earthshot import mon_stats
from earthshot import water_viz as vis
from earthshot import normalize as norm
from earthshot import water_common as common
from statistics import mean

import geemap.eefolium as geemap
import folium
from folium import plugins

import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
ee.Initialize()

In [55]:
def NDVI_func(image):
    date_img = image.get('system:time_start')
    NDVI = ((image.select('B6')).subtract(image.select('B5'))).divide((image.select('B6')).add(image.select('B5')))
    NDVI = NDVI.set({'system:time_start': date_img}).rename('NDVI')
    return NDVI

In [56]:
Landsat_1 = ee.ImageCollection('LANDSAT/LM01/C01/T1').filterDate('1972-07-23', '1975-01-21').map(NDVI_func)
Landsat_2 = ee.ImageCollection('LANDSAT/LM02/C01/T1').filterDate('1975-01-22', '1978-03-04').map(NDVI_func)
Landsat_3 = ee.ImageCollection('LANDSAT/LM03/C01/T1').filterDate('1978-03-05', '1982-07-15').map(NDVI_func)
Landsat_4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_8DAY_NDVI").filterDate('1982-07-16', '1984-02-29').select('NDVI')
Landsat_5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_8DAY_NDVI').filterDate('1984-03-01', '1998-12-31').select('NDVI')
Landsat_7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_NDVI').filterDate('1999-01-01', '2013-03-31').select('NDVI')
Landsat_8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate('2013-04-01', '2021-05-31').select('NDVI')

In [57]:
NDVI_data = Landsat_1.merge(Landsat_2)
NDVI_data = NDVI_data.merge(Landsat_3)
NDVI_data = NDVI_data.merge(Landsat_4)
NDVI_data = NDVI_data.merge(Landsat_5)
NDVI_data = NDVI_data.merge(Landsat_7)
NDVI_data = NDVI_data.merge(Landsat_8)

In [58]:
MAR_projects = ee.FeatureCollection('users/amgadellaboudy/Global_MAR_Inventory')

Projects = ['Rooftop Rainwater Harvesting', 'Barriers and Bunds', 'Channel Spreading', 'Ditches and Furrow', 
            'Excess Irrigation', 'Flooding', 'Sand Storage Dams', 'Trenches', 'Infiltration Ponds and Basins', 'Recharge Dam']


MAR_filtered = (MAR_projects.filter('year_opera > 1982').filter('year_opera < 2021')
                .filter(ee.Filter.inList('specific_m', Projects)))

print(MAR_filtered.aggregate_count('specific_m').getInfo())

211


In [59]:
NDVI_years = ee.ImageCollection('users/amgadellaboudy/EVI_year')
NDVI_2020 = (NDVI_data.filter(ee.Filter.date('2020-01-01', '2020-12-31')).reduce(ee.Reducer.mean()))

for i in range(1983,2020):
    NDVI_image = NDVI_data.filter(ee.Filter.date(('{}-01-01'.format(i)), ('{}-12-31'.format(i)))).reduce(ee.Reducer.mean()) #Take vegetation index for given year, average values into one image
    NDVI_diff = (NDVI_2020.subtract(NDVI_image)).set({'Year': i}) #Store NDVI difference into image, set year as property
    NDVI_years = NDVI_years.merge(NDVI_diff) #Collect into image collection

In [60]:
MAR_1983 = MAR_filtered.filter(ee.Filter.eq('year_opera', 1983))  #Initialize MAR Feature Collection for year 1973
NDVI_1983 = NDVI_years.filter(ee.Filter.eq('Year', 1983)).first() #Take image for NDVI diff for 1973
NDVI_1983_feat = NDVI_1983.sampleRegions(collection = MAR_1983, properties = ['year_opera'], scale = 60, geometries = True) #Initiate image collection for EVI diff at MAR locations
 
for i in range(1984,2020):
    MAR_year = MAR_filtered.filter(ee.Filter.eq('year_opera', i))
    NDVI_set = NDVI_years.filter(ee.Filter.eq('Year', i)).first()
    NDVI_feat = NDVI_set.sampleRegions(collection = MAR_year, properties = ['year_opera'], scale = 60, geometries = True)
    NDVI_1983_feat = NDVI_1983_feat.merge(NDVI_feat)

In [61]:
def success(feature):
    success_feat = feature.set({'Success': 1})
    return success_feat

def fail(feature):
    fail_feat = feature.set({'Success': 0})
    return fail_feat

NDVI_success = NDVI_1983_feat.filter(ee.Filter.gt('NDVI_mean', 0)).map(success)
NDVI_fail = NDVI_1983_feat.filter(ee.Filter.lte('NDVI_mean', 0)).map(fail)
NDVI_data = NDVI_success.merge(NDVI_fail)

In [62]:
#import groundwater and aquifer information in
groundwater_1 = ee.FeatureCollection('users/amgadellaboudy/groundwater_info')
groundwater_2 = ee.FeatureCollection('users/amgadellaboudy/groundwater_info_2')
aquifer = ee.FeatureCollection('users/amgadellaboudy/aquifer_characteristics')

def missing_values(image):
    mask = image.gte(0)
    image_masked = image.updateMask(mask)
    return image_masked

water_area_img= groundwater_1.reduceToImage(properties = ['waterarea'], reducer = ee.Reducer.first())
total_renew_water_img= groundwater_1.reduceToImage(properties = ['totrenewab'], reducer = ee.Reducer.first())
arid_land_img= groundwater_1.reduceToImage(properties = ['extofaridl'], reducer = ee.Reducer.first())
humid_land_img= groundwater_1.reduceToImage(properties = ['extofhumid'], reducer = ee.Reducer.first())
gw_recharge_img= groundwater_2.reduceToImage(properties = ['annualgwre'], reducer = ee.Reducer.first())
aquifer_without_img = aquifer.reduceToImage(properties = ['extofareaswithoutsignificantaq'], reducer = ee.Reducer.first())
volcanic_aquifer_img = aquifer.reduceToImage(properties = ['extofconsolidatedvolcanicaquif'], reducer = ee.Reducer.first())
fissured_aquifer_img = aquifer.reduceToImage(properties = ['extoffissuredaquifers'], reducer = ee.Reducer.first())
intergranular_aquifer_img = aquifer.reduceToImage(properties = ['extofintergranularaquifers'], reducer = ee.Reducer.first())
metamorphic_aquifer_img = aquifer.reduceToImage(properties = ['extofintrusiveormetamorphicroc'], reducer = ee.Reducer.first())
limestone_aquifer_img = aquifer.reduceToImage(properties = ['extoflimestonedolomiteaquifers'], reducer = ee.Reducer.first())
sandgravel_aquifer_img = aquifer.reduceToImage(properties = ['extofsandgravelaquifers'], reducer = ee.Reducer.first())
sandstone_aquifer_img = aquifer.reduceToImage(properties = ['extofsandstoneaquifers'], reducer = ee.Reducer.first())
unconsolidated_volcnanic_aquifer_img = aquifer.reduceToImage(properties = ['extofunconsolidatedvolcanicaqu'], reducer = ee.Reducer.first())
kastifiedzones_img = aquifer.reduceToImage(properties = ['occofkarstifiedzonesincarbonat'], reducer = ee.Reducer.first())


In [63]:
#Import variables in (Slope, Porosity, Runoff, Soil Types, Annual Precipitation, water accumulation)
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
#Scale = 90 m

smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = (smap_usda_clim
                  .filter(ee.Filter.eq('band', 'avail_porosity_mm')))
avail_porosity_img = avail_porosity.sum()

runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')
runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)
runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum().multiply(720)

precip_clim = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01")
precip_img = precip_clim.reduce('sum')
#https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_CLM_CLM_PRECIPITATION_SM2RAIN_M_v01#bands
#Scale = 1 km

soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

top_soils = soil_types.expression('soil_0 + soil_10 + soil_30',
                             {'soil_0': soil_types.select('b0'),
                             'soil_10': soil_types.select('b10'),
                             'soil_30': soil_types.select('b30')})

bottom_soils = soil_types.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_types.select('b60'),
                                  'soil_100': soil_types.select('b100'),
                                  'soil_200': soil_types.select('b200')})

#Scale = 250 m


water_img = ee.Image('WWF/HydroSHEDS/15ACC').select('b1')
#Measure of water accumulation: https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_15ACC
#Scale = 462 m

organic_matter = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").reduce(ee.Reducer.sum())
#measure of total organic matter in soil, from 0 to 200 m soil depth
#https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_ORGANIC-CARBON_USDA-6A1C_M_v02

In [64]:
#Collect images into one image, convert into feature collection that includes location and type of MAR interventions

img_1 = slope_img.addBands(avail_porosity_img)
img_2 = img_1.addBands(runoff_img)
img_3 = img_2.addBands(precip_img)
img_4 = img_3.addBands(top_soils)
img_5 = img_4.addBands(bottom_soils)
img_6 = img_5.addBands(water_area_img)
img_7 = img_6.addBands(total_renew_water_img)
img_8 = img_7.addBands(arid_land_img)
img_9 = img_8.addBands(humid_land_img)
img_10 = img_9.addBands(gw_recharge_img)
img_11 = img_10.addBands(aquifer_without_img)
img_12 = img_11.addBands(volcanic_aquifer_img)
img_13 = img_12.addBands(fissured_aquifer_img)
img_14 = img_13.addBands(intergranular_aquifer_img)
img_15 = img_14.addBands(metamorphic_aquifer_img)
img_16 = img_15.addBands(limestone_aquifer_img)
img_17 = img_16.addBands(sandgravel_aquifer_img)
img_18 = img_17.addBands(sandstone_aquifer_img)
img_19 = img_18.addBands(unconsolidated_volcnanic_aquifer_img)
img_20 = img_19.addBands(organic_matter)
img_21 = img_20.addBands(kastifiedzones_img)
training_img = img_21.addBands(water_img)

print(training_img.bandNames().getInfo())

['slope', 'avail_porosity_mm', 'surface_runoff', 'sum', 'b0', 'b60', 'first', 'first_1', 'first_2', 'first_3', 'first_4', 'first_5', 'first_6', 'first_7', 'first_8', 'first_9', 'first_10', 'first_11', 'first_12', 'first_13', 'sum_1', 'first_14', 'b1']


In [65]:
target = 'Success'
bands = ['slope', 'avail_porosity_mm', 'surface_runoff', 'sum', 'b0', 'b60', 'first', 'first_1', 'first_2', 'first_3', 'first_4', 'first_5', 'first_6', 'first_7', 'first_8', 'first_9', 'first_10', 'first_11', 'first_12', 'first_13', 'sum_1', 'first_14', 'b1']

training = training_img.sampleRegions(collection= NDVI_data, properties = [target], scale = 60, geometries = True)

In [66]:
print(training.size().getInfo())

173


In [67]:
print(NDVI_data.size().getInfo())

206


In [68]:
#Test/ validation split, fit classifier for training data

split = 0.7
training = training.randomColumn()

training_split = training.filter(ee.Filter.lt('random', split))
validation_split = training.filter(ee.Filter.gte('random', split))

classifier = ee.Classifier.smileGradientTreeBoost(1000).train(features= training_split, classProperty= target, inputProperties = bands)

In [69]:
#Create predicted MAR intervention image and validation data

classified = training_split.classify(classifier)
validated = validation_split.classify(classifier)

In [70]:
#Training accuracy

trainAccuracy = classified.errorMatrix('Success', 'classification')
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

Resubstitution error matrix:  [[39, 0], [0, 80]]
Training overall accuracy:  1


In [71]:
#Validation accuracy

validationAccuracy = validated.errorMatrix('Success', 'classification')
print('Validation error matrix: ', validationAccuracy.getInfo())
print('Validation overall accuracy: ', validationAccuracy.accuracy().getInfo())

Validation error matrix:  [[11, 5], [9, 29]]
Validation overall accuracy:  0.7407407407407407


In [72]:
classified_img = training_img.classify(classifier)

In [188]:
NDVI_image = NDVI_data.filter(ee.Filter.date('2000-01-01', '2000-12-31')).reduce(ee.Reducer.mean()) #Take vegetation index for given year, average values into one image
NDVI_diff = (NDVI_2020.subtract(NDVI_image)) #Store NDVI difference into image, set year as property

In [73]:
the_map = geemap.Map()

Landsat = ee.ImageCollection('LANDSAT/LM03/C01/T1')

vis_params = {
    'min': 0, 'max': 1, 'dimensions': 512,
    'palette': ['Red', 'Blue']}

the_map.addLayer(classified_img, vis_params, 'Classification')
the_map.addLayer(MAR_1983.draw(color= '006600', pointRadius = 5, strokeWidth= 5), {}, name = 'MAR locations')

vis.folium_display(the_map)